In [319]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import json


pd.set_option('display.max_columns', None)

In [173]:
# # Import and join data sets

# folder = '../data/'
# written = False

# def data_import(folder_path):
#     global written
#     files = os.listdir(folder_path)
#     output_file = '../data/taxi.csv'
#     for file in files:
#         if len(file) <= 9:
#             input_file = os.path.join(folder_path, file)
#             try:
#                 df = pd.read_csv(input_file, encoding='utf-8')
#             except UnicodeDecodeError:
#                 try:
#                     df = pd.read_csv(input_file, encoding='latin1')
#                 except UnicodeDecodeError:
#                     try:
#                         df = pd.read_csv(input_file, encoding='ISO-8859-1')
#                     except Exception as e:
#                         print(f"Error reading file {file}: {e}")
#                         continue
#             if not written:
#                 df.to_csv(output_file, index=False)
#                 written = True
#             else:
#                 df.to_csv(output_file, index=False, header=False, mode='a')

# data_import(folder)

In [174]:
#Reading in dataframe of 2022 trips
trips = pd.read_csv('../data/taxi.csv')
trips.head()

,Unnamed: 0,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,base_passenger_fare,tips,driver_pay,congestion_surcharge
0,0.0,11/02/2022 08:12:02 PM,11/02/2022 08:14:51 PM,11/02/2022 08:15:05 PM,11/02/2022 08:23:56 PM,66.0,97.0,1.42,8.86,0.0,6.34,0.00
1,1.0,11/02/2022 08:12:02 PM,11/02/2022 08:13:53 PM,11/02/2022 08:14:04 PM,11/02/2022 08:35:08 PM,144.0,140.0,5.49,28.24,0.0,17.54,2.75
2,2.0,11/02/2022 08:12:03 PM,11/02/2022 08:14:05 PM,11/02/2022 08:15:41 PM,11/02/2022 08:24:20 PM,36.0,80.0,1.02,6.54,0.0,5.76,0.00
3,3.0,11/02/2022 08:12:03 PM,11/02/2022 08:13:53 PM,11/02/2022 08:15:45 PM,11/02/2022 08:38:16 PM,211.0,141.0,3.91,23.43,0.0,20.82,2.75
4,4.0,11/02/2022 08:12:03 PM,11/02/2022 08:14:32 PM,11/02/2022 08:15:31 PM,11/02/2022 08:20:42 PM,85.0,188.0,0.89,7.12,0.0,5.87,0.00


In [175]:
#Dropping Unnamed: 0 column and checking dtypes
trips = trips.drop(columns=['Unnamed: 0'])
trips.dtypes

request_datetime         object
on_scene_datetime        object
pickup_datetime          object
dropoff_datetime         object
PULocationID            float64
DOLocationID            float64
trip_miles              float64
base_passenger_fare     float64
tips                    float64
driver_pay              float64
congestion_surcharge    float64
dtype: object

In [176]:
#Checking for df shape and NaNs
print(trips.shape)
trips.isna().sum()

(3964992, 11)


request_datetime              4
on_scene_datetime       1096172
pickup_datetime               4
dropoff_datetime              4
PULocationID                  4
DOLocationID                  4
trip_miles                    4
base_passenger_fare           4
tips                          4
driver_pay                    4
congestion_surcharge          4
dtype: int64

In [177]:
#Checking the 4 missing for each
trips[trips['request_datetime'].isna()]

,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,base_passenger_fare,tips,driver_pay,congestion_surcharge
170000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2152496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2152497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [178]:
#Dropping the 4
trips = trips.drop([170000, 170001, 2152496, 2152497])
trips.isna().sum()

request_datetime              0
on_scene_datetime       1096168
pickup_datetime               0
dropoff_datetime              0
PULocationID                  0
DOLocationID                  0
trip_miles                    0
base_passenger_fare           0
tips                          0
driver_pay                    0
congestion_surcharge          0
dtype: int64

In [179]:
#Looking at NaN values of on_scene_datetime
trips[trips['on_scene_datetime'].isna()]

,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,base_passenger_fare,tips,driver_pay,congestion_surcharge
8,11/02/2022 08:12:03 PM,NaN,11/02/2022 08:15:19 PM,11/02/2022 08:46:49 PM,229.0,129.0,5.820,22.88,5.73,23.54,2.75
11,11/02/2022 08:12:03 PM,NaN,11/02/2022 08:16:21 PM,11/02/2022 08:33:54 PM,50.0,113.0,2.559,13.62,0.00,12.32,2.75
13,11/02/2022 08:12:03 PM,NaN,11/02/2022 08:17:08 PM,11/02/2022 08:35:11 PM,34.0,61.0,3.181,20.38,0.00,13.31,0.00
16,11/02/2022 08:12:04 PM,NaN,11/02/2022 08:15:34 PM,11/02/2022 08:49:43 PM,125.0,135.0,16.983,38.86,0.00,37.93,2.75
17,11/02/2022 08:12:04 PM,NaN,11/02/2022 08:13:43 PM,11/02/2022 08:19:07 PM,106.0,189.0,0.743,7.86,0.00,5.47,0.00
...,...,...,...,...,...,...,...,...,...,...,...
3964976,02/28/2022 11:13:46 AM,NaN,02/28/2022 11:16:23 AM,02/28/2022 12:09:06 PM,237.0,95.0,9.722,33.70,0.00,37.28,2.75
3964980,02/28/2022 11:13:46 AM,NaN,02/28/2022 11:18:58 AM,02/28/2022 11:26:05 AM,56.0,173.0,1.064,7.45,0.00,5.47,0.00
3964983,02/28/2022 11:13:47 AM,NaN,02/28/2022 11:14:57 AM,02/28/2022 11:36:07 AM,166.0,48.0,5.771,21.49,0.00,17.04,2.75
3964988,02/28/2022 11:13:47 AM,NaN,02/28/2022 11:17:34 AM,02/28/2022 11:32:09 AM,140.0,170.0,2.938,15.35,2.97,10.59,2.75


In [180]:
trips['request_datetime'] = pd.to_datetime(trips['request_datetime'], format='mixed')
trips['req_index'] = pd.to_datetime(trips['request_datetime']).dt.date
trips['req_time'] = pd.to_datetime(trips['request_datetime']).dt.time

In [181]:
#trips['req_date'] = pd.to_datetime(trips['req_date'])
trips.columns

Index(['request_datetime', 'on_scene_datetime', 'pickup_datetime',
       'dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_miles',
       'base_passenger_fare', 'tips', 'driver_pay', 'congestion_surcharge',
       'req_index', 'req_time'],
      dtype='object')

In [182]:
#Setting index as date & sorting by index
trips.set_index('req_index', inplace=True)
trips = trips.sort_index()

#Creating req_date column again so can merge later
trips['req_date'] = pd.to_datetime(trips['request_datetime']).dt.date
trips['req_date'] = pd.to_datetime(trips['req_date'], format='mixed')
trips.head()

,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,base_passenger_fare,tips,driver_pay,congestion_surcharge,req_time,req_date
req_index,,,,,,,,,,,,,
2022-01-04,2022-01-04 07:32:25,NaN,01/04/2022 07:38:25 AM,01/04/2022 07:53:23 AM,183.0,20.0,4.148,17.69,4.95,12.13,0.00,07:32:25,2022-01-04
2022-01-04,2022-01-04 07:15:54,01/04/2022 07:17:11 AM,01/04/2022 07:17:45 AM,01/04/2022 07:29:10 AM,262.0,238.0,1.630,10.11,0.00,7.68,2.75,07:15:54,2022-01-04
2022-01-04,2022-01-04 07:15:54,NaN,01/04/2022 07:23:33 AM,01/04/2022 07:32:34 AM,50.0,161.0,1.751,10.05,2.00,7.85,2.75,07:15:54,2022-01-04
2022-01-04,2022-01-04 07:15:54,NaN,01/04/2022 07:23:11 AM,01/04/2022 07:28:24 AM,124.0,124.0,1.633,10.76,0.00,5.47,0.00,07:15:54,2022-01-04
2022-01-04,2022-01-04 07:15:54,01/04/2022 07:17:54 AM,01/04/2022 07:19:39 AM,01/04/2022 07:38:50 AM,180.0,63.0,6.070,24.41,0.00,16.33,0.00,07:15:54,2022-01-04


In [183]:
#Converting pickup time to datetime and creating a just pickup time column
trips['pickup_datetime'] = pd.to_datetime(trips['pickup_datetime'], format='mixed')
trips['pickup_time'] = pd.to_datetime(trips['pickup_datetime']).dt.time

#Converting dropoff time to datetime and creating a just pickup time column
trips['dropoff_datetime'] = pd.to_datetime(trips['dropoff_datetime'], format='mixed')
trips['dropoff_date'] = pd.to_datetime(trips['dropoff_datetime']).dt.date
trips['dropoff_time'] = pd.to_datetime(trips['dropoff_datetime']).dt.time

#Checking it worked
trips.head(3)

,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,base_passenger_fare,tips,driver_pay,congestion_surcharge,req_time,req_date,pickup_time,dropoff_date,dropoff_time
req_index,,,,,,,,,,,,,,,,
2022-01-04,2022-01-04 07:32:25,NaN,2022-01-04 07:38:25,2022-01-04 07:53:23,183.0,20.0,4.148,17.69,4.95,12.13,0.00,07:32:25,2022-01-04,07:38:25,2022-01-04,07:53:23
2022-01-04,2022-01-04 07:15:54,01/04/2022 07:17:11 AM,2022-01-04 07:17:45,2022-01-04 07:29:10,262.0,238.0,1.630,10.11,0.00,7.68,2.75,07:15:54,2022-01-04,07:17:45,2022-01-04,07:29:10
2022-01-04,2022-01-04 07:15:54,NaN,2022-01-04 07:23:33,2022-01-04 07:32:34,50.0,161.0,1.751,10.05,2.00,7.85,2.75,07:15:54,2022-01-04,07:23:33,2022-01-04,07:32:34


In [184]:
trips.head()

,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,base_passenger_fare,tips,driver_pay,congestion_surcharge,req_time,req_date,pickup_time,dropoff_date,dropoff_time
req_index,,,,,,,,,,,,,,,,
2022-01-04,2022-01-04 07:32:25,NaN,2022-01-04 07:38:25,2022-01-04 07:53:23,183.0,20.0,4.148,17.69,4.95,12.13,0.00,07:32:25,2022-01-04,07:38:25,2022-01-04,07:53:23
2022-01-04,2022-01-04 07:15:54,01/04/2022 07:17:11 AM,2022-01-04 07:17:45,2022-01-04 07:29:10,262.0,238.0,1.630,10.11,0.00,7.68,2.75,07:15:54,2022-01-04,07:17:45,2022-01-04,07:29:10
2022-01-04,2022-01-04 07:15:54,NaN,2022-01-04 07:23:33,2022-01-04 07:32:34,50.0,161.0,1.751,10.05,2.00,7.85,2.75,07:15:54,2022-01-04,07:23:33,2022-01-04,07:32:34
2022-01-04,2022-01-04 07:15:54,NaN,2022-01-04 07:23:11,2022-01-04 07:28:24,124.0,124.0,1.633,10.76,0.00,5.47,0.00,07:15:54,2022-01-04,07:23:11,2022-01-04,07:28:24
2022-01-04,2022-01-04 07:15:54,01/04/2022 07:17:54 AM,2022-01-04 07:19:39,2022-01-04 07:38:50,180.0,63.0,6.070,24.41,0.00,16.33,0.00,07:15:54,2022-01-04,07:19:39,2022-01-04,07:38:50


In [185]:
#Importing Weather dataframe
weather = pd.read_csv('../data/nyc_weather_2022.csv')
weather.head(5)

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,precipprob,precipcover,preciptype,snow,snowdepth,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,11206,2022-01-01,56.4,50.0,52.8,56.4,50.0,52.8,50.2,90.7,0.722,100,54.17,rain,0.0,0.0,13.2,9.0,135.7,1008.3,99.9,5.3,14.4,1.2,1,NaN,2022-01-01T07:19:55,2022-01-01T16:39:09,0.96,"Rain, Overcast",Cloudy skies throughout the day with rain.,rain,"72505394728,KLGA,F8726,KNYC,72503014732"
1,11206,2022-01-02,58.9,38.9,50.4,58.9,31.8,48.8,45.2,83.1,0.096,100,12.50,rain,0.0,0.0,30.7,15.3,311.2,1004.7,92.7,7.2,24.3,2.2,2,NaN,2022-01-02T07:20:00,2022-01-02T16:40:00,0.00,"Rain, Overcast",Cloudy skies throughout the day with rain clea...,rain,"72505394728,KLGA,F8726,KNYC,72503014732"
2,11206,2022-01-03,38.4,24.0,30.4,32.3,12.2,20.8,13.6,49.7,0.000,0,0.00,"rain,snow",0.7,0.4,27.9,17.2,359.5,1019.3,69.5,9.9,11.9,1.0,1,NaN,2022-01-03T07:20:03,2022-01-03T16:40:53,0.03,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"72505394728,KLGA,F8726,KNYC,72503014732"
3,11206,2022-01-04,34.5,20.0,27.6,30.9,7.1,20.2,10.4,48.7,0.000,0,0.00,NaN,0.0,0.2,24.2,13.1,284.6,1028.0,17.5,9.9,59.5,5.2,3,NaN,2022-01-04T07:20:04,2022-01-04T16:41:47,0.06,Clear,Clear conditions throughout the day.,clear-day,"72505394728,KLGA,F8726,KNYC,72503014732"
4,11206,2022-01-05,47.0,31.4,39.7,45.0,24.9,35.6,32.6,76.1,0.234,100,33.33,"rain,snow",0.0,0.0,20.4,12.8,216.7,1014.0,73.9,8.3,27.0,2.3,1,NaN,2022-01-05T07:20:02,2022-01-05T16:42:43,0.10,"Snow, Rain, Partially cloudy",Partly cloudy throughout the day with rain or ...,rain,"72505394728,KLGA,F8726,KNYC,72503014732"


In [186]:
#Dropping columsn I don't need
wth = weather.drop(columns=['name', 'tempmax', 'tempmin', 'feelslikemax',
       'feelslikemin', 'feelslike', 'dew', 'humidity', 'precip', 'precipprob',
       'precipcover', 'icon', 'snow', 'snowdepth', 'windgust',
       'windspeed', 'winddir', 'sealevelpressure', 'cloudcover', 'visibility',
       'solarradiation', 'solarenergy', 'uvindex', 'severerisk', 'sunrise',
       'sunset', 'moonphase', 'conditions', 'description', 'stations'])
wth.head()

,datetime,temp,preciptype
0,2022-01-01,52.8,rain
1,2022-01-02,50.4,rain
2,2022-01-03,30.4,"rain,snow"
3,2022-01-04,27.6,NaN
4,2022-01-05,39.7,"rain,snow"


In [187]:
wth['datetime'] = pd.to_datetime(wth['datetime'], format='mixed')
wth.dtypes

datetime      datetime64[ns]
temp                 float64
preciptype            object
dtype: object

In [188]:
wth['preciptype'].value_counts()

preciptype
rain                          183
rain,snow                      18
snow                           11
rain,freezingrain,snow          2
rain,freezingrain,snow,ice      2
rain,freezingrain               2
freezingrain                    1
freezingrain,snow,ice           1
rain,snow,ice                   1
Name: count, dtype: int64

In [189]:
wth['preciptype'].isna().sum()

144

In [190]:
#Ordinal Mapping preciptypes
wth['preciptype'] = wth['preciptype'].fillna('missing')
precip = {'missing': 0, 'rain': 1, 'snow': 2, 'rain,snow': 3, 
          'rain,freezingrain,snow': 4, 'rain,freezingrain,snow,ice': 5,
          'rain,freezingrain': 6, 'freezingrain': 6, 'freezingrain,snow,ice': 5, 'rain,snow,ice': 5} 
wth['preciptype'] = wth['preciptype'].map(precip)
wth['preciptype'].value_counts()

preciptype
1    183
0    144
3     18
2     11
5      4
6      3
4      2
Name: count, dtype: int64

In [191]:
wth.rename(columns={'datetime': 'req_date'}, inplace=True)
wth['req_date'] = pd.to_datetime(wth['req_date'], format='mixed')

In [192]:
wth

,req_date,temp,preciptype
0,2022-01-01,52.8,1
1,2022-01-02,50.4,1
2,2022-01-03,30.4,3
3,2022-01-04,27.6,0
4,2022-01-05,39.7,3
...,...,...,...
360,2022-12-27,31.7,0
361,2022-12-28,40.1,0
362,2022-12-29,45.0,0
363,2022-12-30,50.8,0


In [193]:
wth.req_date.value_counts(), trips.req_date.value_counts()

(req_date
 2022-01-01    1
 2022-09-09    1
 2022-09-07    1
 2022-09-06    1
 2022-09-05    1
              ..
 2022-05-01    1
 2022-04-30    1
 2022-04-29    1
 2022-04-28    1
 2022-12-31    1
 Name: count, Length: 365, dtype: int64,
 req_date
 2022-08-09    40000
 2022-08-03    40000
 2022-08-13    40000
 2022-08-11    40000
 2022-08-06    40000
               ...  
 2022-04-15     4988
 2022-08-07     2788
 2022-11-13     2690
 2022-04-03     1346
 2022-09-18      800
 Name: count, Length: 198, dtype: int64)

In [194]:
#Importing Taxi Zones
zones = pd.read_csv('../data/taxi_zones.csv')
zones.head()

,OBJECTID,Shape_Leng,the_geom,Shape_Area,zone,LocationID,borough
0,1,0.116357,MULTIPOLYGON (((-74.18445299999996 40.69499599...,0.000782,Newark Airport,1,EWR
1,2,0.433470,MULTIPOLYGON (((-73.82337597260663 40.63898704...,0.004866,Jamaica Bay,2,Queens
2,3,0.084341,MULTIPOLYGON (((-73.84792614099985 40.87134223...,0.000314,Allerton/Pelham Gardens,3,Bronx
3,4,0.043567,MULTIPOLYGON (((-73.97177410965318 40.72582128...,0.000112,Alphabet City,4,Manhattan
4,5,0.092146,MULTIPOLYGON (((-74.17421738099989 40.56256808...,0.000498,Arden Heights,5,Staten Island


In [195]:
#Dropping columns I don't need
zones = zones.drop(columns=['OBJECTID', 'Shape_Leng', 'Shape_Area'])
zones.head(5)

,the_geom,zone,LocationID,borough
0,MULTIPOLYGON (((-74.18445299999996 40.69499599...,Newark Airport,1,EWR
1,MULTIPOLYGON (((-73.82337597260663 40.63898704...,Jamaica Bay,2,Queens
2,MULTIPOLYGON (((-73.84792614099985 40.87134223...,Allerton/Pelham Gardens,3,Bronx
3,MULTIPOLYGON (((-73.97177410965318 40.72582128...,Alphabet City,4,Manhattan
4,MULTIPOLYGON (((-74.17421738099989 40.56256808...,Arden Heights,5,Staten Island


In [196]:
#Casting LocationID as float and changing name to match trips dataframe
zones['LocationID'] = zones['LocationID'].astype(float)
zones = zones.rename(columns={'LocationID':'PULocationID'})

In [197]:
trips.PULocationID.value_counts(), zones.PULocationID.value_counts()

(PULocationID
 138.0    74630
 132.0    70696
 79.0     57320
 61.0     53120
 230.0    52884
          ...  
 111.0      132
 99.0        66
 2.0         12
 110.0        6
 199.0        4
 Name: count, Length: 260, dtype: int64,
 PULocationID
 103.0    3
 56.0     2
 1.0      1
 175.0    1
 181.0    1
         ..
 93.0     1
 94.0     1
 95.0     1
 96.0     1
 262.0    1
 Name: count, Length: 260, dtype: int64)

trips|wth **left on** req_date

trips|zones **left on** PULocationID

In [198]:
trips.shape, wth.shape

((3964988, 16), (365, 3))

In [199]:
#Merging trips and weather
df = pd.merge(
    left=trips.reset_index(),
    right=wth,
    how='left',
    on='req_date'
)
trips.shape, df.shape

((3964988, 16), (3964988, 19))

In [200]:
df

,req_index,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,base_passenger_fare,tips,driver_pay,congestion_surcharge,req_time,req_date,pickup_time,dropoff_date,dropoff_time,temp,preciptype
0,2022-01-04,2022-01-04 07:32:25,NaN,2022-01-04 07:38:25,2022-01-04 07:53:23,183.0,20.0,4.148,17.69,4.95,12.13,0.00,07:32:25,2022-01-04,07:38:25,2022-01-04,07:53:23,27.6,0
1,2022-01-04,2022-01-04 07:15:54,01/04/2022 07:17:11 AM,2022-01-04 07:17:45,2022-01-04 07:29:10,262.0,238.0,1.630,10.11,0.00,7.68,2.75,07:15:54,2022-01-04,07:17:45,2022-01-04,07:29:10,27.6,0
2,2022-01-04,2022-01-04 07:15:54,NaN,2022-01-04 07:23:33,2022-01-04 07:32:34,50.0,161.0,1.751,10.05,2.00,7.85,2.75,07:15:54,2022-01-04,07:23:33,2022-01-04,07:32:34,27.6,0
3,2022-01-04,2022-01-04 07:15:54,NaN,2022-01-04 07:23:11,2022-01-04 07:28:24,124.0,124.0,1.633,10.76,0.00,5.47,0.00,07:15:54,2022-01-04,07:23:11,2022-01-04,07:28:24,27.6,0
4,2022-01-04,2022-01-04 07:15:54,01/04/2022 07:17:54 AM,2022-01-04 07:19:39,2022-01-04 07:38:50,180.0,63.0,6.070,24.41,0.00,16.33,0.00,07:15:54,2022-01-04,07:19:39,2022-01-04,07:38:50,27.6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3964983,2022-12-31,2022-12-31 04:39:08,NaN,2022-12-31 04:45:18,2022-12-31 05:08:43,129.0,228.0,14.602,35.75,0.00,29.44,0.00,04:39:08,2022-12-31,04:45:18,2022-12-31,05:08:43,49.8,1
3964984,2022-12-31,2022-12-31 04:39:09,12/31/2022 04:44:44 AM,2022-12-31 04:45:21,2022-12-31 05:17:02,234.0,71.0,12.930,54.73,0.00,33.48,2.75,04:39:09,2022-12-31,04:45:21,2022-12-31,05:17:02,49.8,1
3964985,2022-12-31,2022-12-31 04:39:09,12/31/2022 04:45:50 AM,2022-12-31 04:47:52,2022-12-31 05:14:02,131.0,265.0,21.430,58.15,0.00,44.73,0.00,04:39:09,2022-12-31,04:47:52,2022-12-31,05:14:02,49.8,1
3964986,2022-12-31,2022-12-31 04:39:06,NaN,2022-12-31 04:43:36,2022-12-31 05:07:54,51.0,163.0,17.206,44.92,0.00,32.94,2.75,04:39:06,2022-12-31,04:43:36,2022-12-31,05:07:54,49.8,1


In [201]:
df = pd.merge(
    left=df,
    right=zones,
    how='left',
    on='PULocationID'
)

In [202]:
df.shape

(3975858, 22)

In [253]:
df.isna().sum()

req_index                     0
request_datetime              0
on_scene_datetime       1099258
pickup_datetime               0
dropoff_datetime              0
PULocationID                  0
DOLocationID                  0
trip_miles                    0
base_passenger_fare           0
tips                          0
driver_pay                    0
congestion_surcharge          0
req_time                      0
req_date                      0
pickup_time                   0
dropoff_date                  0
dropoff_time                  0
temp                          0
preciptype                    0
the_geom                    888
zone                        888
borough                     888
dtype: int64

In [256]:
na_zone = df[df['zone'].isna()]

In [259]:
na_zone['PULocationID'].value_counts()

PULocationID
57.0     700
265.0    188
Name: count, dtype: int64

In [269]:
zones.iloc[262]

the_geom        MULTIPOLYGON (((-73.94383256699986 40.78285908...
zone                                               Yorkville East
PULocationID                                                262.0
borough                                                 Manhattan
Name: 262, dtype: object

**Discovery**:
After heavy evaluation, there are no taxi zones 57.0 and 265.0, resulting in 888 NaNs.

In [279]:
df = df.reset_index()
df

,req_index,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,base_passenger_fare,tips,driver_pay,congestion_surcharge,req_time,req_date,pickup_time,dropoff_date,dropoff_time,temp,preciptype,the_geom,zone,borough
0,2022-01-04,2022-01-04 07:32:25,NaN,2022-01-04 07:38:25,2022-01-04 07:53:23,183.0,20.0,4.148,17.69,4.95,12.13,0.00,07:32:25,2022-01-04,07:38:25,2022-01-04,07:53:23,27.6,0,MULTIPOLYGON (((-73.83676643699985 40.84338596...,Pelham Bay,Bronx
1,2022-01-04,2022-01-04 07:15:54,01/04/2022 07:17:11 AM,2022-01-04 07:17:45,2022-01-04 07:29:10,262.0,238.0,1.630,10.11,0.00,7.68,2.75,07:15:54,2022-01-04,07:17:45,2022-01-04,07:29:10,27.6,0,MULTIPOLYGON (((-73.94383256699986 40.78285908...,Yorkville East,Manhattan
2,2022-01-04,2022-01-04 07:15:54,NaN,2022-01-04 07:23:33,2022-01-04 07:32:34,50.0,161.0,1.751,10.05,2.00,7.85,2.75,07:15:54,2022-01-04,07:23:33,2022-01-04,07:32:34,27.6,0,MULTIPOLYGON (((-73.99667705784736 40.77351005...,Clinton West,Manhattan
3,2022-01-04,2022-01-04 07:15:54,NaN,2022-01-04 07:23:11,2022-01-04 07:28:24,124.0,124.0,1.633,10.76,0.00,5.47,0.00,07:15:54,2022-01-04,07:23:11,2022-01-04,07:28:24,27.6,0,MULTIPOLYGON (((-73.850964537 40.6717195109998...,Howard Beach,Queens
4,2022-01-04,2022-01-04 07:15:54,01/04/2022 07:17:54 AM,2022-01-04 07:19:39,2022-01-04 07:38:50,180.0,63.0,6.070,24.41,0.00,16.33,0.00,07:15:54,2022-01-04,07:19:39,2022-01-04,07:38:50,27.6,0,MULTIPOLYGON (((-73.8411960569998 40.683351365...,Ozone Park,Queens
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3975853,2022-12-31,2022-12-31 04:39:08,NaN,2022-12-31 04:45:18,2022-12-31 05:08:43,129.0,228.0,14.602,35.75,0.00,29.44,0.00,04:39:08,2022-12-31,04:45:18,2022-12-31,05:08:43,49.8,1,MULTIPOLYGON (((-73.87586503899996 40.76605165...,Jackson Heights,Queens
3975854,2022-12-31,2022-12-31 04:39:09,12/31/2022 04:44:44 AM,2022-12-31 04:45:21,2022-12-31 05:17:02,234.0,71.0,12.930,54.73,0.00,33.48,2.75,04:39:09,2022-12-31,04:45:21,2022-12-31,05:17:02,49.8,1,MULTIPOLYGON (((-73.98996936399989 40.73490456...,Union Sq,Manhattan
3975855,2022-12-31,2022-12-31 04:39:09,12/31/2022 04:45:50 AM,2022-12-31 04:47:52,2022-12-31 05:14:02,131.0,265.0,21.430,58.15,0.00,44.73,0.00,04:39:09,2022-12-31,04:47:52,2022-12-31,05:14:02,49.8,1,MULTIPOLYGON (((-73.77732562899998 40.72239270...,Jamaica Estates,Queens
3975856,2022-12-31,2022-12-31 04:39:06,NaN,2022-12-31 04:43:36,2022-12-31 05:07:54,51.0,163.0,17.206,44.92,0.00,32.94,2.75,04:39:06,2022-12-31,04:43:36,2022-12-31,05:07:54,49.8,1,MULTIPOLYGON (((-73.82091617199997 40.86891693...,Co-Op City,Bronx


In [284]:
df['req_index'] = pd.to_datetime(df['req_index'])
df = df.set_index('req_index')
df = df.sort_index()

In [313]:
df2 = df.drop(columns=['request_datetime', 'on_scene_datetime',	'pickup_datetime', 'dropoff_datetime', 'the_geom'])

In [315]:
df2['borough'] = df2['borough'].fillna('missing')
ordinals = {'missing': 0, 'Bronx': 1, 'Manhattan': 2, 'Queens': 3, 
          'Brooklyn': 4, 'Staten Island': 5} 
df2['borough'] = df2['borough'].map(ordinals)

In [316]:
df2['req_time'] = pd.to_datetime(df2['req_time'], format='%H:%M:%S')
df2['pickup_time'] = pd.to_datetime(df2['pickup_time'], format='%H:%M:%S')
df2['dropoff_date'] = pd.to_datetime(df2['dropoff_date'], format='mixed')
df2['dropoff_time'] = pd.to_datetime(df2['dropoff_time'], format='%H:%M:%S')

df2.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3975858 entries, 2022-01-04 to 2022-12-31
Data columns (total 16 columns):
 #   Column                Dtype         
---  ------                -----         
 0   PULocationID          float64       
 1   DOLocationID          float64       
 2   trip_miles            float64       
 3   base_passenger_fare   float64       
 4   tips                  float64       
 5   driver_pay            float64       
 6   congestion_surcharge  float64       
 7   req_time              datetime64[ns]
 8   req_date              datetime64[ns]
 9   pickup_time           datetime64[ns]
 10  dropoff_date          datetime64[ns]
 11  dropoff_time          datetime64[ns]
 12  temp                  float64       
 13  preciptype            int64         
 14  zone                  object        
 15  borough               int64         
dtypes: datetime64[ns](5), float64(8), int64(2), object(1)
memory usage: 760.9 MB


In [334]:
df2.to_csv('../data/taxi2022.csv')